In [1]:
# from google.colab import drive
# drive.mount('/content/drive')  # use this if you are using google colab

In [2]:
from sklearn.cluster import KMeans
import pickle, os
from scipy.spatial import distance
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

In [3]:
def load_dataset(name_file):

    desired_directory = '.' # Replace with your desired directory path

    file_path = os.path.join(desired_directory, name_file) 

    with open(file_path, 'rb') as f:
        data_dict = pickle.load(f)

    return data_dict


In [4]:
loaders_dict = load_dataset(f"dataset-flowers102-features.pkl")
x_train = loaders_dict["x_train"]
x_test = loaders_dict["x_test"]
y_train = loaders_dict["y_train"]
y_test = loaders_dict["y_test"]
print(f"x_train:{x_train.shape}, y_train:{y_train.shape}")
print(f"x_test:{x_test.shape}, y_test:{y_test.shape}")


x_train:(4094, 512), y_train:(4094,)
x_test:(4095, 512), y_test:(4095,)


## clustering images

In [5]:
k_number = 50
kmeans = KMeans(n_clusters=k_number)
clusters = kmeans.fit_predict(x_train)
centroids = kmeans.cluster_centers_

c:\Users\ewint\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


## find nearest clusters neighbors

### find nearest clusters and their indices

In [7]:
def find_nearest_clusters_neighbors(x_train, x_test, k):
    knn_classifier = KNeighborsClassifier(n_neighbors=k)
    knn_classifier.fit(x_train, range(len(x_train)))
    nearest_indices = knn_classifier.kneighbors(x_test, n_neighbors=k, return_distance=False)
    # nearest_neighbors = x_train[nearest_indices]
    # nearest_neighbors = [x_train[indices] for indices in nearest_indices]

    return nearest_indices

### get the data of nearest clusters

In [10]:
def gather_clusters_data(k_number, clusters):
    clusters_data = {i: [] for i in range(k_number)}
    clusters_data_labels = {i: [] for i in range(k_number)}
    for i, label in enumerate(clusters):
        clusters_data[label].append(x_train[i])
        clusters_data_labels[label].append(y_train[i])
    return clusters_data, clusters_data_labels

In [12]:
def get_nearest_clusters_data(nearest_clusters_indices, clusters_data, clusters_data_labels):
    # Access data points in the cluster with centroid index 'nearest_clusters_indices'
    all_data_in_nearest_clusters = []
    all_labels_in_nearest_clusters = []
    for indices in nearest_clusters_indices:
        data_in_nearest_clusters = []
        labels_in_nearest_clusters = []
        for index in indices:
            data_in_nearest_clusters.extend(clusters_data[index])
            labels_in_nearest_clusters.extend(clusters_data_labels[index])
        all_data_in_nearest_clusters.append(data_in_nearest_clusters)
        all_labels_in_nearest_clusters.append(labels_in_nearest_clusters)
    return all_data_in_nearest_clusters, all_labels_in_nearest_clusters

## classify using data in nearest clusters

In [21]:
def classify_knn(x_train, y_train, x_test, y_test, k):
    y_preds = []
    for data_train, label_train, one_data_test in zip(x_train, y_train, x_test):
        knn_classifier = KNeighborsClassifier(n_neighbors=k)
        knn_classifier.fit(data_train, label_train)
        y_preds.append(knn_classifier.predict([one_data_test]))
    
    accuracy = accuracy_score(y_test, y_preds)
    score = ( accuracy * 100 ) + ( k * (-0.2))
    return y_preds, accuracy, score

In [23]:
print(f"number of predicted labels: {len(predictions)}")
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"score: {score}")
# print(f"Predicted labels:")
# for i in range(len(predictions)):
#     print(f"{predictions[i]} --> {y_test[i]}")

number of predicted labels: 4095
Accuracy: 85.18%
score: 83.97704517704517


In [ ]:
# find nearest klisters neighbors----------------------------------
k_nearest_clusters = 7
x_train_clusters = centroids
nearest_clusters_indices = find_nearest_clusters_neighbors(x_train_clusters, 
                                                           x_test, 
                                                           k=k_nearest_clusters)
# gather clusters data---------------------------------------------
clusters_data, clusters_data_labels = gather_clusters_data(k_number, clusters)
# get nearest clusters data----------------------------------------
all_data_in_nearest_clusters, all_labels_in_nearest_clusters = get_nearest_clusters_data(nearest_clusters_indices, 
                                                                                 clusters_data, 
                                                                                 clusters_data_labels)
# classify test data using knn-------------------------------------
k_nearest_data = 6
x_train_data = all_data_in_nearest_clusters
y_train_data = all_labels_in_nearest_clusters
predictions, accuracy, score = classify_knn(x_train_data, y_train_data, x_test, y_test, k=k_nearest_data)
#---------------------------------------------
